### Run LAMMPS with Python

The motivation is because Python is awesome, which makes scripting and coding much easier and more flexible.

To run LAMMPS with Python, it is required to first install Python and compile LAMMPS as a shared library. For compilation we are going to use Intel Compilers (alternatively one can also use GNU compilers). 

**0. Install Python**

The simplest way to install Python is using [Anaconda](https://www.anaconda.com/products/individual#Downloads), which is a free scientific Python distribution for Linux, Mac, and Windows. Follow the installation instructions for those different platforms.

**1. Install Intel Compilers**

The Intel compilers are freely available on Linux systems. First download and install the following:
- [Intel Fortran Compiler](https://software.intel.com/content/www/us/en/develop/articles/oneapi-standalone-components.html#fortran)
- [Intel C++ Compiler](https://software.intel.com/content/www/us/en/develop/articles/oneapi-standalone-components.html#inpage-nav-5-undefined)
- [Intel MPI libraries](https://software.intel.com/content/www/us/en/develop/articles/oneapi-standalone-components.html#inpage-nav-10-undefined)
- [Intel MKL libraries](https://software.intel.com/content/www/us/en/develop/articles/oneapi-standalone-components.html#inpage-nav-11-5)

Then we need to setup the Intel environment variables by adding the line
`source /home/user/intel/oneapi/setvars.sh` to the `.bashrc` file.

**2. Install FFTW Package**

- tar xzvf fftw-3.3.9.tar.gz
- mv fftw-3.3.9 fftw
- cd fftw
- ./configure --prefix=/home/user/fftw/ \ <br/>
  CC=icc F77=ifort \ # specify C and Fortran compilers <br/>
  --enable-shared --enable-static \ # build shared and static libs <br/>
  --enable-avx --enable-avx2 --enable-fma --enable-sse2 \ # add various optimizations <br/>
  --enable-mpi --enable-threads --enable-openmp \ # build MPI and OpenMP parallelizations <br/>
- make -j 4 # number of cores = 4
- make install

**3. Install LAMMPS Code**

- tar xzvf lammps-stable.tar.gz
- mv lammps-29Oct20 lammps
- cd lammps/src/
- vim MAKE/Makefile.mpi: Specify MPI and FFTW libraries as follows:
    ```
        MPI_INC =       -I/home/user/intel/oneapi/mpi/2021.2.0/include
        MPI_PATH =      -L/home/user/intel/oneapi/mpi/2021.2.0/lib
        MPI_LIB =       -lmpi
    ```
    ```
        FFT_INC =       -I/home/user/fftw/include
        FFT_PATH =      -L/home/user/fftw/lib
        FFT_LIB =       -lfftw3
    ```
    
- make yes-std # install standard packages
- make no-lib # uninstall packages that require extra libs
- make mpi mod=shlib # parallel build, compile as a shared lib

A successful compilation should produce in the `lammps/src/` folder `lmp_mpi`, `liblammps.a`, `liblammps_mpi.so`, and `liblammps.so`. 
    
**4. Interface LAMMPS to Python**

In order to interface LAMMPS to Python smoothly, we need to setup the environment variables by adding the following lines to the `.bashrc` file:

```
export PYTHONPATH=/home/user/lammps/python:$PYTHONPATH
export LD_LIBRARY_PATH=/home/user/lammps/src:$LD_LIBRARY_PATH
```

Now let's confirm if LAMMPS is interfaced to Python...

```
$ python
Python 3.6.13 |Anaconda, Inc.| (default, Jun  4 2021, 14:25:59) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from lammps import lammps
>>> lmp = lammps()
LAMMPS (29 Oct 2020)
>>> 
```

It looks fine. In next tutorial we will perform some real calculations by running LAMMPS with Python.